In [17]:
import sys
import os


from include.gpt_caller import GPTCaller


In [24]:
caller=GPTCaller()

with open('./prompts/distillation/system_prompt', 'r') as file:
    system_prompt = file.read()

print(system_prompt)

user_prompt=["where can i find love."]

response=caller.create_prompt(user_prompt_list=user_prompt,system_prompt_list=system_prompt)



You are an AI assistant, your task is to help me extract key word and key task from a long instruction. I will give you a few examples first. VERY IMPORTANT:the keyword MUST be a simple non-abstract object, it must be well-defined and common (e.g. car, mouse, apple, keyboard, traffic light)!!! It MUST be easy to detect with computer vision algorithms. You can infer what the keyword is based on the intention of the instruction. You must respond with in format of {"keyword":[list of key word],"key instruction":[list of key instruction]}

Examples:
1. Instruction: can you help me find my phone?
Response: {"keyword":["phone"],"key instruction":["find the phone"]}

2. Instruction: I am thristy.
Response:{"keyword":["water bottle"],"key instruction":["find a water bottle"]}

3. Instruction: I'm at a crosswalk and I need to cross the street.
Response:{"keyword":["traffic light button"],"key instruction":["find the traffic light button"]}

4. Instruction: I need to open the door.
Response:{"ke

In [25]:
caller.call()

'{"keyword":["heart"],"key instruction":["find a heart symbol"]}'